In [ ]:
import numpy as np
from scipy import signal as sig
import librosa
from src.synth import Synth
import matplotlib.pyplot as plt



def three_step_envelope(signal, K=200, freq_cutoff=0.125, padlen=150):
    # Take the absolute value of the signal
    abs_signal = np.abs(signal)

    # Split signal into K bunches of N/K samples and take the max
    # value from each bunch
    N = len(abs_signal)
    peaks = []
    for i in range(0, N, K):
        peaks.append(np.max(abs_signal[i:i+K]))
    peaks = np.asarray(peaks)

    # Apply a LPF with a cut off of 0.125 Times the Niquist (125 Hz)
    b, a = sig.butter(8, freq_cutoff)
    lpf_peaks = sig.filtfilt(b, a, peaks, padlen=padlen)
    
    return lpf_peaks

def find_first_onset(signal, K=200):
    onsets = []
    while len(onsets) == 0: 
        approx_env = three_step_envelope(signal=signal, K=K)
        # convert envelope to Fortran-contiguous
        ftrn_env = np.asfortranarray(approx_env) 

        onsets = librosa.onset.onset_detect(y=ftrn_env, sr=sr)
        print(onsets)
        K-=10
    
    return onsets[0]

def attack(signal, onset, sr):
    '''
    Computes attack time by taking the difference 
    of the argmax and onset and dividing by the sampling rate
    '''
    return (np.argmax(np.abs(signal)) - onset)/sr

def extract_envelope(signal, sr):
    '''
    Approximate envelope, Extract attack
    '''
    K = 200
    approx_env = three_step_envelope(signal=signal, K=K)
    # convert envelope to Fortran-contiguous
    ftrn_env = np.asfortranarray(approx_env) 
    
    first_onset = find_first_onset(signal)
    attack_t = attack(ftrn_env, first_onset, sr)
    
    return {
        "attack": attack_t,
        "decay": 0.5,
        "sustain": (1.0, 0.5),
        "release": 2.0,
    }

In [ ]:
import soundfile as sf
from src.synth import Synth


fname = '../datasets/guitar/open-e.wav'
(signal, sr) = sf.read(fname)

synth = Synth(signal, sr)
synth.set_srate(sr)
env = synth.extract_envelope()

In [ ]:
import ipywidgets as widgets

# Initialize sliders
attack = widgets.FloatSlider(
    description='attack:',
    value=env['attack'],
    min=0,
    max=2.0,
    step=0.00001,
    readout_format='.4f',
)
decay = widgets.FloatSlider(
    description='decay:',
    value=env['decay'],
    min=0,
    max=2.0,
    step=0.00001,
    readout_format='.4f',
)
sustain_time = widgets.FloatSlider(
    description='sustain time:',
    value=env['sustain'][0],
    min=0,
    max=10,
    step=0.00001,
    readout_format='.4f',
)
sustain_power = widgets.FloatSlider(
    description='sustain power:',
    value=env['sustain'][1],
    min=0,
    max=1.0,
    step=0.00001,
    readout_format='.4f',
)
release = widgets.FloatSlider(
    description='release:',
    value=env['release'],
    min=0,
    max=10,
    step=0.00001,
    readout_format='.4f',
)
ui = widgets.VBox([attack, decay, sustain_time, sustain_power, release])

def f(attack, decay, sustain_time, sustain_power, release):
    env = {
        "attack": attack,
        "decay": decay,
        "sustain": (sustain_time, sustain_power),
        "release": release
    }
    synth.set_envelope(env)
    print(synth.get_output())
    # construct envelope

out = widgets.interactive_output(
    f, {'attack': attack, 'decay': decay, 'sustain_time': sustain_time,
        'sustain_power': sustain_power, 'release': release}
)

display(ui, out)
display(ui, out)

In [ ]:
plt.plot(synth.envelope_amp_track()[0])

In [ ]:
freqs = synth.set_frequencies()
osc_freq_widgets = [ 
    widgets.FloatSlider(
        description=f"Osc{i} Freq:",
        value=freq[0],
        min=40,
        max=10000,
        step=10,
    )
    for (i, freq) in enumerate(freqs)
]

osc_amp_widgets = [ 
    widgets.FloatSlider(
        description=f"Osc{i} Amp:",
        value=freq[1],
        min=0,
        max=100,
        step=10,
    )
    for (i, freq) in enumerate(freqs)
]

osc_wave_widgets = [ 
    widgets.Dropdown(
        options=['sinusoid', 'triangle', 'square'],
        description=f"Osc{i} Wave Type:",
        value='sinusoid',
    )
    for (i, freq) in enumerate(freqs)
]

def osc_callback(index, freq, amp, wave_type):
    oscillator_data = {'freq': freq, 'amp': amp, 'wave_type': wave_type, 'phase': 0}
    print(index, oscillator_data)

for i in range(len(osc_freq_widgets)):
    freq_widget = osc_freq_widgets[i]
    amp_widget = osc_amp_widgets[i]
    wave_widget = osc_wave_widgets[i]
    
    osc_ui = widgets.VBox([freq_widget, amp_widget, wave_widget])
    osc_out = widgets.interactive_output(
        osc_callback, 
        {'index': widgets.IntText(value=i), 'freq':freq_widget, 'amp': amp_widget, 'wave_type': wave_widget}
    )
    display(osc_ui, osc_out)